In [5]:
from sqlalchemy import create_engine, MetaData, Table
import pandas as pd
import uuid
import json

# Replace these with your actual PostgreSQL credentials
user = "hrm"
password = "hrm"
host = "35.223.205.197"  # IP address or hostname
port = "5432"  # Default PostgreSQL port
database = "attendance_tracking"

user= "postgres"
password= "123456"
host= "localhost"
port= "5432"
database= "attendance_tracking"

# Database connection URI for PostgreSQL
pg_database_uri = f'postgresql://{user}:{password}@{host}:{port}/{database}'
pg_engine = create_engine(pg_database_uri)

# SQLite database URI
sqlite_database_uri = 'sqlite:///local_attendance_tracking.db'
sqlite_engine = create_engine(sqlite_database_uri)

# Create MetaData instance for PostgreSQL
pg_metadata = MetaData()
pg_metadata.reflect(bind=pg_engine)

# Specify the tables you want to copy
tables_to_copy = ['departments','workshops','user_images', 'user_details', 'tracking_history', 'tracking_history_error']

# Iterate over specified tables and copy schema and data to SQLite
for table_name in tables_to_copy:
    if table_name in pg_metadata.tables:
        print(f"Copying table: {table_name}")
        pg_table = pg_metadata.tables[table_name]

        # Fetch data from PostgreSQL table
        data = pd.read_sql_table(table_name, pg_engine)

        # Check if the DataFrame is not empty before processing
        if not data.empty:
            # Convert UUID columns to strings and handle complex data
            for col in data.columns:
                if data[col].dtype == 'object':
                    # Convert UUIDs to strings
                    if len(data[col]) > 0 and isinstance(data[col].iloc[0], uuid.UUID):
                        data[col] = data[col].astype(str)

                    # Convert dictionaries or JSON-like objects to strings
                    elif isinstance(data[col].iloc[0], dict):
                        data[col] = data[col].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)

        # Write data to SQLite table
        data.to_sql(table_name, sqlite_engine, if_exists='replace', index=False)

print("Database copy complete.")

Copying table: departments
Copying table: workshops
Copying table: user_images
Copying table: user_details
Copying table: tracking_history
Copying table: tracking_history_error
Database copy complete.


In [6]:
#check db
import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, Table, select
import psycopg2

# Connect to PostgreSQL database
pg_engine = create_engine('sqlite:///local_attendance_tracking.db')
online_connection = pg_engine.connect()
online_metadata = db.MetaData()

user_details = db.Table('user_details', online_metadata, autoload_with=pg_engine)
user_images = db.Table('user_images', online_metadata, autoload_with=pg_engine)
tracking_history = db.Table('tracking_history', online_metadata, autoload_with=pg_engine)
tracking_history_error = db.Table('tracking_history_error', online_metadata, autoload_with=pg_engine)

In [8]:
# Truy vấn dữ liệu từ bảng
select_query = db.select(user_details)
result_proxy = online_connection.execute(select_query)
result_set = result_proxy.fetchall()

# Hiển thị kết quả
for row in result_set:
    print(row)

('02ea6111-d521-420c-ac62-707ca00b2757', 'f1d2dc5d-a04f-425f-8e23-424c2fbfa406', 'NT10863', None, None, 'Lâm Thị Trang', '0001-01-01', 5, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '0001-01-01', None, None, None, None, None, None, None, None, None, '0001-01-01', '0001-01-01', '0001-01-01', None, None, None, None, None, None, 0, 6, '0001-01-01', 0, None, None, None)
('173af05b-58f3-4756-8d75-b54ca78e03dc', 'dbd23c14-0445-4f00-ba8d-569f3a33ee97', 'NT05255', None, None, 'Phùng Bá Huy', '0001-01-01', 5, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '0001-01-01', None, None, None, None, None, None, None, None, None, '0001-01-01', '0001-01-01', '0001-01-01', None, None, None, None, None, None, 0, 6, '0001-01-01', 0, None, None, None)
('1943bf5d-9b62-4443-b948-ed56e9167519', 'dbd23c14-0445-4f00-ba8d-569f3a33ee97', 'NT11185', None, None, 'Vũ Thị Thảo', '0001-01-01', 5, None, None, None, None, None, None, None, None,